In [2]:
!git clone https://github.com/Sarah627/Credit-Card-Fraud-Detection.git

Cloning into 'Credit-Card-Fraud-Detection'...


In [3]:
import pandas as pd

In [12]:
dataset = pd.read_csv(r"C:\Users\ahmed\Credit-Card-Fraud-Detection\cleansed_dataset.csv")

In [13]:
dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-0.692570,0.684420,-0.681072,0.661313,-0.956476,0.132650,0.162292,0.345529,-0.037786,0.116353,...,-0.132785,-0.130266,0.156352,0.470278,0.074176,-1.042174,-0.050446,-0.045423,-0.334524,0
1,1.271345,0.157499,-0.634418,-0.055032,1.255515,-0.425593,0.997861,0.783638,0.041772,-1.102214,...,0.309792,0.521508,1.952374,-0.022174,0.742493,1.105254,-0.202731,-0.095967,3.822321,1
2,0.752553,-0.221098,0.269237,0.306200,-1.026833,0.280679,0.148012,0.316023,-0.218214,0.189374,...,0.800190,-0.182205,-0.258031,-0.886093,1.945438,-0.037087,-0.310553,-0.401827,-0.344800,0
3,1.011078,0.493430,-0.263709,-0.186122,0.256494,-0.080800,-0.219538,0.294556,0.056499,0.080358,...,0.190881,0.884429,0.873441,1.715494,-0.744589,0.373816,0.105638,-0.138772,1.045192,1
4,0.648476,0.821269,-0.478579,0.173912,-0.449114,0.672798,0.199015,0.594452,-0.115222,0.412175,...,-0.092216,0.283036,-0.184909,-2.491057,0.834222,-0.909566,-0.111027,-0.250955,-0.222291,0


In [16]:
fraudData = dataset[dataset['Class'] == 1]
nonFraudData = dataset[dataset['Class'] == 0]
print(fraudData)
print(nonFraudData)


         Time        V1        V2        V3        V4        V5        V6  \
1    1.271345  0.157499 -0.634418 -0.055032  1.255515 -0.425593  0.997861   
3    1.011078  0.493430 -0.263709 -0.186122  0.256494 -0.080800 -0.219538   
6   -1.430992 -2.353613  1.533878 -2.490417  2.804992 -1.260919 -0.819810   
8   -0.036072 -0.095959  0.054684  0.480650  0.253219 -0.050459  0.499552   
14  -0.405513  0.312936 -0.534543  0.531705 -1.128157 -0.080589  0.770698   
..        ...       ...       ...       ...       ...       ...       ...   
972  1.186107 -0.506308 -0.421749 -0.308604  0.042227  0.533933 -0.751871   
973  0.749112  0.086178 -2.491779 -0.090482 -0.301469 -0.117554  0.304698   
978 -1.664678  0.508720  0.525976 -0.341147  1.191842  0.736497 -1.114748   
979 -0.958669 -1.086204  1.169410 -1.313648  1.398395 -1.559302 -1.706891   
980  1.687507  0.200748  0.246465 -0.137852  0.042447 -0.030150  0.421334   

           V7        V8        V9  ...       V21       V22       V23  \
1  

In [37]:
import tensorflow as tf
from tensorflow.keras import layers, models


In [38]:
def generator(input_data):
    model=models.Sequential([
        #first layer
        layers.Dense(128, activation = 'relu', input_dim = input_data), 
        layers.Dense(256, activation = 'relu'),
        layers.Dense(512, activation = 'relu'),
        layers.Dense(30, activation = 'tanh')
        
    ])
    return model
    


In [39]:
def discriminator(input_data):
    model=models.Sequential([
        #first layer
        layers.Dense(512, activation = 'relu', input_dim = input_data), 
        layers.Dense(256, activation = 'relu'),
        layers.Dense(128, activation = 'relu'),
        layers.Dense(1, activation = 'sigmoid')
        
    ])
    return model

In [42]:
def com_gan(generator, discriminator):
    discriminator.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0005), loss = "binary_crossentropy",metrics =['accuracy'])
    discriminator.trainable = False
    input_Gan=layers.Input(shape=(100,))
    generated_data = generator(input_Gan)
    output = discriminator(generated_data)
    gan = tf.keras.Model(input_Gan, output )
    gan.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss="binary_crossentropy")
    return gan

In [45]:
generator_model = generator(input_data = 100)
discriminator_model = discriminator(input_data = 30)
ganNetwork = com_gan(generator_model, discriminator_model)

In [47]:
import numpy as np
from sklearn.model_selection import train_test_split

In [48]:
xFraudData = fraudData.drop(columns=['Class'])
yFraudData = fraudData['Class']


In [49]:
X_train, X_test, y_train, y_test = train_test_split(xFraudData, yFraudData, test_size=0.3, random_state=42)


In [51]:
epochs = 1000
batch_size = 16
for epoch in range(epochs):
 
    real = xFraudData[np.random.randint(0, xFraudData.shape[0], batch_size)]
    rlabels = np.ones((batch_size, 1))
    noise = np.random.normal(0, 1, (batch_size, 100))
    fake = generator.predict(noise)
    flabels = np.zeros((batch_size, 1))
    d_loss_real = discriminator.train_on_batch(real, rlabels)
    d_loss_fake = discriminator.train_on_batch(fake, flabels)
    noise = np.random.normal(0, 1, (batch_size, 100))
    gan_labels = np.ones((batch_size, 1)) 
    g_loss = gan.train_on_batch(noise, gan_labels)
    
    if epoch % 100 == 0:
        print(f"{epoch} [D loss real: {d_loss_real[0]}, acc.: {d_loss_real[1]}] [D loss fake: {d_loss_fake[0]}] [G loss: {g_loss}]")
    

KeyError: "None of [Index([267, 294, 43, 487, 47, 148, 255, 77, 462, 249, 472, 296, 56, 152, 72,\n       11],\n      dtype='int32')] are in the [columns]"